In [252]:
import re
import string

import markdown
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame

# merge the train and test csvs

train_csv = pd.concat([pd.read_csv('train/good.csv'), pd.read_csv('train/bad.csv')], axis=0).reset_index()
test_csv = pd.concat([pd.read_csv('test/good.csv'), pd.read_csv('test/bad.csv')], axis=0).reset_index()
train_csv
# pd.read_csv('test/bad.csv').shape[0]
# pd.read_csv('test/bad.csv').columns

,index,id,readme,is_good
0,0,1,# Babylon.js\r\n\r\nGetting started? Play dire...,True
1,1,2,"<div align=""center"">\r\n <a href=""https://w...",True
2,2,3,<!--# \xe4\xbb\x8a\xe5\xb9\xb4\xe6\x9c\x80\xe5...,True
3,3,4,"<p align=""center"">\n <a href=""https://get.fou...",True
4,4,5,Metasploit [![Build Status](https://travis-ci....,True
...,...,...,...,...
2560,1070,1071,# -\n=\xe6\x8a\x8a\xe4\xbd\xa0\xe5\xb8\xa6\xe5...,False
2561,1071,1072,"# Hi, I\'m Kyle, a full stack javascript devel...",False
2562,1072,1073,# hello-world\nHello world stuff.\n\nHello thi...,False
2563,1073,1074,# \xe7\xa2\x8c\xe6\x9f\x9a\xe5\x8f\xb6\xe7\x9a...,False


In [239]:



def add_size_columns(csv):
    lengths = []
    for i, row in csv.iterrows():
        lengths.append(len(str(row.readme)))
    csv.insert(2, 'length', lengths)
    return csv


add_size_columns(train_csv)
add_size_columns(test_csv)


# contains 'id', 'readme', 'is_good'

,index,id,length,readme,is_good
0,0,1,4024,# Beego [![Test](https://github.com/beego/beeg...,True
1,1,2,3545,"<p align=""center""><a href=""https://laravel.com...",True
2,2,3,6061,# ![TodoMVC](media/logo.png)\n\n> Helping you ...,True
3,3,4,28301,# Sortable &nbsp; [![Financial Contributors on...,True
4,4,5,13584,"<p align=""center"">\n <a href=""#""><img src=""./...",True
...,...,...,...,...,...
2591,1139,1140,93,# 50projects_50days\n 50 unique mini-projects ...,False
2592,1140,1141,405,# KMG-CAFE\nFIRST PROJECTS\nThis project based...,False
2593,1141,1142,37,# srs\nAutomating services to users\n,False
2594,1142,1143,16,# htmlsitetest\n,False


In [244]:
def create_vocabulary(preprocessed_dataframe: DataFrame) -> set:
    vocabulary = set()
    for i, row in preprocessed_dataframe.iterrows():
        for j in row.readme:
            vocabulary.add(j)
    return vocabulary


In [241]:
def preprocess_html(html):
    # Use BeautifulSoup to parse the HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Remove any script or style tags
    for tag in soup(['script', 'style']):
        tag.decompose()

    # Remove HTML tags and attributes
    text = soup.get_text()
    text = re.sub(r'<[^>]*>', '', text)
    text = text.replace('/', ' ').replace('\\n', ' ').replace('\\r', ' ')
    text = re.sub(r'\\x..', '', text)
    text = text.translate(text.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    text = text.lower()

    # Split the text into words
    words = text.split()
    if len(words) >= 1 and words[0] == 'b':
        del words[0]

    # Return the preprocessed text
    return words


count = 0


def preprocess_markdown(markdown_text):
    markdown_text = str(markdown_text)
    html = markdown.markdown(markdown_text)
    # Preprocess the HTML
    words = preprocess_html(html)
    return words


def preprocess(csv):
    a = csv['readme'].apply(preprocess_markdown)
    return a


train_csv['readme'] = preprocess(train_csv)

train_csv
# test_csv = preprocess(test_csv)
# for i, row in train_csv.iterrows():
#     print(len(row.readme))
#     break


0       [babylon, js, getting, started, play, directly...
1       [java, pencil2, computer, cloud, art, floppy, ...
2       [banner, 2, 0, ui, banner, 2, 0, banner, 1, 4,...
3       [install, documentation, releases, contributin...
4       [metasploit, the, metasploit, framework, is, r...
                              ...                        
2560                                                   []
2561    [hi, i, m, kyle, a, full, stack, javascript, d...
2562    [hello, world, hello, world, stuff, hello, thi...
2563                                                   []
2564    [hello, wrold, just, another, repository, gith...
Name: readme, Length: 2565, dtype: object

In [242]:
test_csv['readme'] = preprocess(test_csv)

test_csv


0       [beego, beego, is, used, for, rapid, developme...
1       [about, laravel, note, this, repository, conta...
2       [helping, you, select, an, mv, framework, webs...
3       [sortable, sortable, is, a, javascript, librar...
4       [a, completely, customizable, framework, for, ...
                              ...                        
2591    [50projects, 50days, 50, unique, mini, project...
2592    [kmg, cafe, first, projects, this, project, ba...
2593               [srs, automating, services, to, users]
2594                                       [htmlsitetest]
2595    [hello, world, first, time, repository, hey, i...
Name: readme, Length: 2596, dtype: object

In [257]:
vocabulary = create_vocabulary(train_csv)
vocabulary
# train_readmes = train_csv
# test_readmes = test_csv

{'m680x',
 'malicious',
 'maddi',
 'lerp',
 'tle',
 'neuecc',
 'oam',
 'tlsharp',
 'chandrasekaran',
 'corrective',
 'myadapter',
 'bindkey',
 'ppassetsactioncontroller',
 'retief',
 'midin',
 'elgentos',
 '879',
 'asanovic',
 'emmet',
 'clogs',
 'subcommand',
 '4bn1',
 'leftover',
 'kkbox',
 'predominantly',
 'leftvalue',
 'jaidedai',
 'mcclellan',
 'krug',
 'jvm',
 'tendr',
 'colorizedstring',
 'z',
 'codenameone',
 'sobjectizer',
 'honcho',
 'tamcore',
 'subshell',
 'minimal',
 'neko',
 '5b',
 '6446657322759488014',
 '1052',
 'portia',
 'privileged',
 'tokuhirom',
 '606',
 'formulations',
 'jdk19webquarkus',
 'platformlessons',
 '45368',
 'cpuguy83',
 'librairies',
 'cyltableviewplaceholder',
 'histbuff',
 'failsafe',
 'juices',
 'gaxatte',
 'hwillson',
 'morestachio',
 'unlink',
 'moderators',
 '2781',
 '2332',
 'nothing',
 'exert',
 'galley',
 'expirementation',
 '3b1',
 'wkentaro',
 'hant',
 'heinemeier',
 'purescript',
 'nextdns',
 'restsharp',
 'nf997',
 'audigier',
 'codebook'

In [258]:
from sklearn.feature_extraction.text import TfidfVectorizer


def input(csv):
    res = list(csv['readme'])
    for i in range(len(res)):
        res[i] = ' '.join(res[i])
    return res


train_vectors = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform(input(train_csv))
test_vectors = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform(input(test_csv))
# print(vectors.toarray())
# vectors = vectorize(good_train_csv['readme'])
# for i in vectors.toarray()[0]:
#     print(i)

In [259]:
import random
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

# print(vectors_train,true_train)
model.fit(train_vectors, train_csv.is_good)
model.score(test_vectors, test_csv.is_good)



0.9356702619414484

In [289]:
test_readme = """

"""


def predict(readme: str) -> bool:
    vector_html = markdown.markdown(readme)
    preprocessed_test = preprocess_html(vector_html)
    vector_test = TfidfVectorizer(vocabulary=vocabulary, min_df=1).fit_transform([' '.join(preprocessed_test)])
    return bool(model.predict(vector_test)[0])


print(predict(test_readme))

False
